In [ ]:
import torch

with open('../nano-gpt-messages.txt', 'r', encoding='utf-8') as f:
    text = f.read()

vocab = sorted(set(text))

vocab_size = len(vocab)

stoi = { ch:i for i,ch in enumerate(vocab) } # dict comperhension eg. A:0
itos = { i:ch for i,ch in enumerate(vocab) } # dict comperhension eg. 0:A

def encode(input: str):
    return [stoi[c] for c in input]

def decode(input: list[int]):
    return ''.join([itos[i] for i in input])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * (len(data)))
train_data = data[:n]
val_data = data[n:]

print(len(data))

import torch.nn as nn
n_embd = 8
block_size = 4

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
    
    def forward(self, idx):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(idx)

        logits = torch.tensor(T, C) # hocu da za svako slovo dobijem sanse za naredno T, C

        loss = 

        return logits, loss
    
    def backward():
        return
    

def generate(input: str):
    idx = encode(input)
    model = BigramLanguageModel(vocab)
    return model.forward()




314937
